In [ ]:
import os
import asyncio
from typing import Awaitable, Callable
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from agent_framework import AgentRunContext
from dotenv import load_dotenv

load_dotenv()
print("✅ Importaciones completadas")

## Implementación del Middleware

In [ ]:
async def logging_agent_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """Middleware simple que registra la ejecución del agente."""
    print("🟢 [INICIO] Agente comenzando ejecución...")

    # Continuar con la ejecución del agente
    await next(context)

    print("🔴 [FIN] Agente finalizó ejecución!")

print("✅ Middleware de logging definido")

## Creación y Ejecución del Agente

In [ ]:
async def main():
    print("🚀 Iniciando agente con middleware...\n")
    
    credential = AzureCliCredential()

    async with AzureAIAgentClient(
        async_credential=credential, 
        project_endpoint=os.getenv("AZURE_PROJECT_ENDPOINT"),
        model_deployment_name=os.getenv("MODEL")
    ).create_agent(
        name="GreetingAgent",
        instructions="Eres un asistente de saludos amigable.",
        middleware=logging_agent_middleware,
    ) as agent:
        result = await agent.run("¡Hola!")
        print(f"\n💬 Respuesta: {result.text}")

await main()

## Middleware Avanzado con Métricas

In [ ]:
import time

async def metrics_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """Middleware que captura métricas de ejecución."""
    start_time = time.perf_counter()
    print(f"📊 [METRICS] Iniciando ejecución del agente...")
    
    try:
        await next(context)
        elapsed = time.perf_counter() - start_time
        print(f"📊 [METRICS] Ejecución completada en {elapsed*1000:.2f}ms")
    except Exception as e:
        elapsed = time.perf_counter() - start_time
        print(f"❌ [METRICS] Falló después de {elapsed*1000:.2f}ms: {e}")
        raise

# Probar con múltiples middlewares
async def test_multiple_middlewares():
    credential = AzureCliCredential()
    
    async with AzureAIAgentClient(
        async_credential=credential, 
        project_endpoint=os.getenv("AZURE_PROJECT_ENDPOINT"),
        model_deployment_name=os.getenv("MODEL")
    ).create_agent(
        name="TestAgent",
        instructions="Eres un asistente útil.",
        middleware=[logging_agent_middleware, metrics_middleware],
    ) as agent:
        result = await agent.run("Cuéntame un dato interesante")
        print(f"\n💬 {result.text}")

await test_multiple_middlewares()

## Análisis y Conclusiones

### Ventajas del Agent Middleware:
1. **Visibilidad completa**: Intercepta toda la ejecución del agente
2. **Métricas globales**: Mide tiempo total incluyendo múltiples llamadas a funciones
3. **Logging centralizado**: Un solo punto para logging de alto nivel
4. **Error handling**: Captura errores en cualquier punto de la ejecución

### Casos de Uso:
- **Monitoreo de performance**: Tracking de latencia end-to-end
- **Auditoría**: Registro de todas las ejecuciones
- **Rate limiting**: Control de frecuencia de uso
- **Authentication**: Verificación de permisos antes de ejecutar